In [0]:
%tensorflow_version 2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import tensorflow_datasets as tfds
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

Adatok letöltése

In [0]:
data = tfds.load("cats_vs_dogs")

A letöltött adatból csak a tanuláshoz szükséges adatot tartjuk meg

In [0]:
train_data = data["train"]

Kiválasztunk 10 képet a halmazból, félre teszünk tesztelési célra, ezeket a hálózat nem fogja megtanulni

In [0]:
test_data =data["train"].take(10)
orig_test_data = data["train"].take(10)
train_data = data["train"].skip(10)

Nézzük meg a képeket, amiken tesztelni szeretnénk

In [0]:
fig, axes = plt.subplots(1, 5, figsize=(20,20))
for d, ax in zip(test_data.take(5), axes):
  ax.imshow(d["image"]) 

Egy méretre vágjuk a képeket, illetve a megfelelő adattípusra konvertáljuk

In [0]:
test_data = test_data.map(lambda img: tf.cast(tf.image.resize_with_crop_or_pad(img["image"], 150,150), tf.int32))
train_data = train_data.map(lambda x: (tf.cast(tf.image.resize_with_crop_or_pad(x["image"], 150,150), tf.int32), x["label"]))


Az adathalmazt megkeverjük és kisebb részegységekre bontjuk

In [0]:
train_data = train_data.shuffle(1024).batch(32)
test_data = test_data.batch(10)
train_data = train_data.prefetch(1)

Létrehozzuk magát a modellt

In [0]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(150, 150 ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

Az elkészült modelt véglegesítjük

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

Elkezdődhet a tanulás

In [0]:
model.fit(train_data, epochs=100)

Saját kép megjelenítése a hálózat által felcímkézve

In [0]:
def predict_own_picture(img_path):
  orig_img = cv2.imread(img_path)
  orig_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
  image = cv2.resize(orig_img, dsize=(150,150))
  image = image.reshape(1, 150, 150, 3)
  pred = model.predict_classes(image, batch_size=None)
  plt.imshow(orig_img)
  if pred == 0:
    plt.title("cat")
  else:
    plt.title("dog")

In [0]:
predict_own_picture()

Teszteljük le az elején különvett 10 képet

In [0]:
predictions = model.predict_classes(test_data, batch_size=None)

Jelenítsük meg az eredményt

In [0]:
fig, axes = plt.subplots(1, 10, figsize=(20,10))
for d, ax, prediction in zip(orig_test_data.take(10), axes, predictions):
  ax.imshow(d["image"])
  if prediction == 0:
    ax.set_title("cat")
  else:
    ax.set_title("dog")